In [4]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.9.19


In [3]:
import pickle
import pandas as pd

In [22]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

In [23]:
from sklearn.pipeline import make_pipeline

In [24]:
import mlflow

In [8]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [1]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename, engine='fastparquet')
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df= read_data('yellow_tripdata_2023-03.parquet')


In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
type(y_pred)

numpy.ndarray

In [12]:
y_pred.shape


(3316216,)

In [13]:
df_pred = pd.DataFrame(y_pred, columns=['prediction'])


In [14]:
df_pred.shape

(3316216, 1)

In [ ]:
pipeline = make_pipeline(
    DictVectorizer(),
    RandomForestRegressor(**params, n_jobs=-1)
)

In [ ]:
with mlflow.start_run():

    params = dict(max_depth=20, n_estimators=100, mon_samples_leaf=10)
    

    mlflow.log_params(params)
    #mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    pipeline.fit(dict_train, y_train)
    
    X_val = dv.transform(dict_val)
    y_pred= model.predict(X_val)
    
    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
    mlflow.sklearn.log_model(model, artifact_path="model")
    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact(dict_vectorizer.bin)
    
    
    

# Q1. Notebook

In [15]:
df_pred.describe()

,prediction
count,3.316216e+06
mean,1.420387e+01
std,6.247490e+00
min,-1.160751e+01
25%,1.096814e+01
50%,1.197354e+01
75%,1.413772e+01
max,7.498748e+01


# Q2. Preparing the output


In [16]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration'],
      dtype='object')

In [17]:
df.shape

(3316216, 20)

In [20]:
year='2023'
month='3'

In [21]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

ValueError: Unknown format code 'd' for object of type 'str'

In [19]:
df['tpep_pickup_datetime'].head()

0   2023-03-01 00:06:43
1   2023-03-01 00:08:25
2   2023-03-01 00:15:04
3   2023-03-01 00:49:37
4   2023-03-01 00:08:04
Name: tpep_pickup_datetime, dtype: datetime64[us]